در ابتدا لایبرری های مورد نیاز را وارد میکنیم
پاندا برای خواندن فایل سی‌اس‌وی
پالپ برای حل برنامه ریزی خطی
و وارنینگ برای عدم نمایش پیام های هشدار

سپس یکسری ورودی از کاربر میگیریم تا بی‌ام‌آر وی را حساب کنیم و از روی آن مقدار کالری مورد نیاز کاربر برای یک روز را حساب کنیم تا به اندازه آن به کاربر غذا پیشنهاد دهیم
مقادیر ورودی بصورت هارد کد وارد شده اند ولی در قسمت کامنت جلوی آن امکان گرفتن ورودی از اینپوت پیاده سازی شده

In [ ]:
# Import all classes of PuLP module
from pulp import *
import pulp as pl
import warnings
warnings.simplefilter(action='ignore')
import pandas

userGender = 'm' #input('Are you male(m) or female(f)?')
userWeight = 80 #float(input('Please input your weight:'))
userActivity = '2' #input('Please input your daily activity from 1(office job or studying) to 3(labor or professional athlete)?')
userBodyFatPercentFactor = 0.95 #Average due to inability to measure
leanFactor = 0.9 #Average for the sake of simplicity
userGenderFactor = float(0.9 if userGender == 'f' else 1)
userActivityFactor = float(1.3 if userActivity == '1' else 1.65 if userActivity == '2' else 2)
userBMR = userWeight * userGenderFactor * 24 * userBodyFatPercentFactor 
userDailyCaloriesNeeded = userBMR * userActivityFactor
print(f'Your (Approximate) BMR is {userBMR} and you need {userDailyCaloriesNeeded} calories daily')
print(f'These are some foods you can take for today:')


: 

در قسمت بعد توسط کتابخانه پاندا فایل دیتای پروژه خوانده می‌شود و در خط قبل آن ست می‌شود که همه سطر ها خوانده شود

In [ ]:

pandas.set_option('display.max_rows', None)
foods = pandas.read_csv('./USDA.csv') #fdc_id, description, food_category_id 

سپس در قسمت بعد عملیات تمیز کردن داده ها انجام می‌شود به این صورت که اگر داده ای با کالری ۰ وجود دارد حذف می‌شود چون باعث خطا در محاسبات برنامه ریزی خطی می شود

In [ ]:

cleanedData = []
for i in range(0,int(len(foods)/10)):
    foodId = foods.iloc[i]['ID']
    foodDescription = foods.iloc[i]['Description']
    calories = foods.iloc[i]['Calories']
    if(calories != 0):
        cleanedData.append({'ID': foodId, 'Description':foodDescription, 'Calories': calories})

print(len(cleanedData))



در قسمت بعد مساله برنامه ریزی خطی تعریف و حل می‌شود
به این صورت که متغیر هایی از جنس اینتیجر و با دامنه بین ۰ و ۱ تعریف می‌وشد
سپس به این متغیر ها ضریب نسبت داده می‌شود که این متغیر ها در ضریبشان جمعشان باید مساوی مقدار کالری روزانه بشود
تا حاصل برابر یک برنامه غذایی روزانه شود

In [ ]:

problem = LpProblem('Diet Problem', LpMaximize)

x = [] #Initialize objective variables
xCoefficients = []


for i in range(0,len(cleanedData)):
    x.append(LpVariable(name= str(cleanedData[i]['ID']), upBound=1, lowBound=0, cat=LpInteger))
    xCoefficients.append(cleanedData[i]['Calories'])

problem += lpSum(x[i] * xCoefficients[i] for i in range(0, len(x)))
problem += lpSum(x[i] * xCoefficients[i] for i in range(0, len(x))) <= userDailyCaloriesNeeded

problem.solve()


بعد از حل حالا نیاز داریم ببینیم کدام متغیر ها یا غذا ها مقدارشان ۱ شده ینی انتخاب شده اند
سپس با آیدی که به اسم متغیر ها اختصاص داده بودیم نام غذا ها را از فایل میخوانیم و به کاربر نشان می‌دهیم

In [ ]:

for v in problem.variables():
    if(v.varValue == 1):
        # print(f'{v.name}')
        food = foods.query(f'ID == {v.name}')
        print(str(food['Description'].values) + '  ' + str(food['Calories'].values))

اگر این فایل به درستی اجرا نشد لطفا با پایتون ورژن ۳ به بالا فایل 
project.py
را اجرا کنید